pKa

MINMAX scaler

In [54]:
import pandas as pd

train = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler_canon_smiles\pKa\train_pka_minmax_only_non_categorizal_features_scaled.csv', index_col=0)

y = train['pKa']
X = train.drop(['fold_id', 'pKa'], axis=1)

test = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler_canon_smiles\pKa\test_pka_minmax_only_non_categorizal_features_scaled.csv', index_col=0)

cv_indices_dict = {0: [], 1: []}
index = 0
for _, row in train.iterrows():
    cv_indices_dict[row['fold_id']].append(index)
    index += 1
cv_indices = [[cv_indices_dict[0], cv_indices_dict[1]], [cv_indices_dict[1], cv_indices_dict[0]]]

In [55]:
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn import datasets
from sklearn import svm

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

def metrics(test, feature):
    y_pred = regr.predict(test.drop([feature], axis=1))
    r2 = r2_score(y_pred=y_pred, y_true = test[feature])
    print(f'=========%s========='%(feature))
    print('R^2 = '+str(round(r2, 3)))
    print('MAE = ', round(mean_absolute_error(y_true = test[feature], y_pred=y_pred), 3))
    print('MSE = ', round(mean_squared_error(y_true = test[feature], y_pred=y_pred), 3))

regr = svm.SVR(kernel='rbf')
scores = cross_validate(regr, X, y, cv=2, groups=train['fold_id'], scoring='r2', return_train_score=True, return_estimator=True)

#metrics(test, 'logP')
regr = scores['estimator'][1]
metrics(test, 'pKa')
metrics(train.drop('fold_id', axis=1), 'pKa')

=========pKa=========
R^2 = 0.317
MAE =  1.647
MSE =  3.914
=========pKa=========
R^2 = 0.387
MAE =  1.534
MSE =  4.042


STANDART scaler

In [56]:
import pandas as pd

train = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler_canon_smiles\pKa\train_pka_standart_scaler_non_categorizal_features_scaled.csv', index_col=0)

y = train['pKa']
X = train.drop(['fold_id', 'pKa'], axis=1)

test = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler_canon_smiles\pKa\test_pka_standart_scaler_non_categorizal_features_scaled.csv', index_col=0)

cv_indices_dict = {0: [], 1: []}
index = 0
for _, row in train.iterrows():
    cv_indices_dict[row['fold_id']].append(index)
    index += 1
cv_indices = [[cv_indices_dict[0], cv_indices_dict[1]], [cv_indices_dict[1], cv_indices_dict[0]]]

In [57]:
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn import datasets
from sklearn import svm

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

def metrics(test, feature):
    y_pred = regr.predict(test.drop([feature], axis=1))
    r2 = r2_score(y_pred=y_pred, y_true = test[feature])
    print(f'=========%s========='%(feature))
    print('R^2 = '+str(round(r2, 3)))
    print('MAE = ', round(mean_absolute_error(y_true = test[feature], y_pred=y_pred), 3))
    print('MSE = ', round(mean_squared_error(y_true = test[feature], y_pred=y_pred), 3))

regr = svm.SVR(kernel='rbf')
scores = cross_validate(regr, X, y, cv=2, groups=train['fold_id'], scoring='r2', return_train_score=True, return_estimator=True)

#metrics(test, 'logP')
regr = scores['estimator'][1]
metrics(test, 'pKa')
metrics(train.drop('fold_id', axis=1), 'pKa')

=========pKa=========
R^2 = 0.662
MAE =  1.094
MSE =  1.938
=========pKa=========
R^2 = 0.637
MAE =  1.094
MSE =  2.39


-------------------

---------------

-----------------

PKA CORRECT CV

In [58]:
from functools import partial
from sklearn import svm
from hyperopt import fmin, tpe, hp

crossvals_list = []

class Trainer:
    def __init__(self) -> None:
        self.c_space = [1, 10, 100, 1000]
        self.space = {
            # 'lr': hp.loguniform('lr', -10, -3),
            'degree': hp.randint("degree", 1, 10),
            'epsilon': hp.loguniform("epsilon", -15, -1),
            'C': hp.choice("C", self.c_space)
        }

        self.PKA_FEATURES = ['RPCS', 'PBF', 'mol_weight', 'dipole_moment', 'PPSA5',
                'avg_atoms_in_cycle', 'nHRing', 'cis/trans', 'FPSA3', 'nF', 'chirality',
                'sasa', 'PNSA5', 'GeomShapeIndex', 'TASA', 'mol_num_cycles',
                'f_freedom', 'nFRing', 'identificator', 'nO', 'nARing', 'nC', 'nFHRing',
                'f_to_fg']

        self.train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler_canon_smiles\pKa\train_pka_standart_scaler.csv', index_col=0)
        self.test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler_canon_smiles\pKa\test_pka_standart_scaler.csv', index_col=0)

        self.find_best_params_with_hyperopt()

    def find_best_params_with_hyperopt(self, ):
        algo = tpe.suggest

        objective_partial = partial(Trainer.optimization_function, X_train=self.train_df)

        best_hyperparams = fmin(fn=objective_partial, space=self.space, algo=algo, max_evals=1000, verbose=1)

        print("Найкращі гіперпараметри:", best_hyperparams)

        X = self.train_df.drop(['pKa', 'fold_id'], axis=1)
        X = X[self.PKA_FEATURES]
        y = self.train_df['pKa']

        regr = svm.SVR(kernel='rbf', degree=best_hyperparams['degree'], epsilon=best_hyperparams['epsilon'], C=self.c_space[best_hyperparams['C']])
        scores_r2 = cross_validate(regr, X, y, cv=2, groups=self.train_df['fold_id'], scoring='r2', return_train_score=True, return_estimator=True)
        scores_mae = cross_validate(regr, X, y, cv=2, groups=self.train_df['fold_id'], scoring='neg_mean_absolute_error', return_train_score=True, return_estimator=True)
        scores_mse = cross_validate(regr, X, y, cv=2, groups=self.train_df['fold_id'], scoring='neg_mean_squared_error', return_train_score=True, return_estimator=True)
        
        train_r2 = scores_r2['train_score'].mean()
        train_mae = scores_mae['train_score'].mean()
        train_mse = scores_mse['train_score'].mean()

        test_r2 = scores_r2['test_score'].mean()
        test_mae = scores_mae['test_score'].mean()
        test_mse = scores_mse['test_score'].mean()

        print(f"CV TRAIN R^2: {train_r2}, MAE: {train_mae}, MSE: {train_mse}")
        print(f"CV TEST R^2: {test_r2}, MAE: {test_mae}, MSE: {test_mse}")

        regr.fit(X, y)
        y_pred_train = regr.predict(X)
        print(Trainer.calculate_metrics(true_values=y, pred_values=y_pred_train))

        return best_hyperparams
    
    @staticmethod
    def calculate_metrics(true_values, pred_values):
        mse = round(mean_squared_error(true_values, pred_values),3)
        mae = round(mean_absolute_error(true_values, pred_values),3)
        r_score = round(r2_score(true_values, pred_values),3)

        return {"mse": mse,
                "mae": mae,
                "r^2": r_score,}

    @staticmethod
    def optimization_function(params, X_train):
        global crossvals_list

        amount_of_cross_vals = 2

        PKA_FEATURES = ['RPCS', 'angle_R1X1R2', 'angle_X2X1R1', 'angle_X1X2R2', 'angle_R2X2R1', 
            'distance_between_atoms_in_cycle_and_f_group', 
            'distance_between_atoms_in_f_group_centers', 'PBF', 'mol_weight', 'dipole_moment', 'PPSA5',
            'avg_atoms_in_cycle', 'nHRing', 'cis/trans', 'FPSA3', 'nF', 'chirality',
            'sasa', 'PNSA5', 'GeomShapeIndex', 'TASA', 'mol_num_cycles',
            'f_freedom', 'nFRing', 'identificator', 'nO', 'nARing', 'nC', 'nFHRing',
            'f_to_fg'
        ]

        X = X_train.drop(['pKa', 'fold_id'], axis=1)
        X = X[PKA_FEATURES]
        y = X_train['pKa']
        
        regr = svm.SVR(kernel='rbf', degree=params['degree'], epsilon=params['epsilon'], C=params['C'])
        scores = cross_validate(regr, X, y, cv=amount_of_cross_vals, groups=X_train['fold_id'], scoring='neg_mean_absolute_error', return_train_score=True, return_estimator=True)
        scores = scores['test_score'].mean()

        # print(params, scores)
        return -scores


Trainer()

  0%|          | 0/1000 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 1000/1000 [00:42<00:00, 23.38trial/s, best loss: 2.265623889457191]
Найкращі гіперпараметри: {'C': 3, 'degree': 6, 'epsilon': 0.36774550650687715}
CV TRAIN R^2: 0.9661229878136655, MAE: -0.2907932431347109, MSE: -0.09863543357356083
CV TEST R^2: -0.4957604433643936, MAE: -2.327562248176942, MSE: -8.373547778030488
{'mse': 0.096, 'mae': 0.289, 'r^2': 0.985}


In [59]:
PKA_FEATURES = ['RPCS', 'angle_R1X1R2', 'angle_X2X1R1', 'angle_X1X2R2', 'angle_R2X2R1', 
            'distance_between_atoms_in_cycle_and_f_group', 
            'distance_between_atoms_in_f_group_centers', 'PBF', 'mol_weight', 'dipole_moment', 'PPSA5',
            'avg_atoms_in_cycle', 'nHRing', 'cis/trans', 'FPSA3', 'nF', 'chirality',
            'sasa', 'PNSA5', 'GeomShapeIndex', 'TASA', 'mol_num_cycles',
            'f_freedom', 'nFRing', 'identificator', 'nO', 'nARing', 'nC', 'nFHRing',
            'f_to_fg'
        ]

train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler_canon_smiles\pKa\train_pka_standart_scaler.csv', index_col=0)
test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler_canon_smiles\pKa\test_pka_standart_scaler.csv', index_col=0)

In [60]:
X = train_df.drop(['pKa', 'fold_id'], axis=1)
X = X[PKA_FEATURES]
y = train_df['pKa']

X_test = test_df.drop(['pKa'], axis=1)
X_test = X_test[PKA_FEATURES]
y_test = test_df['pKa']

In [61]:
regr = svm.SVR(kernel='rbf', degree=6, epsilon=0.36774550650687715, C=1000)
scores = cross_validate(regr, X, y, cv=2, groups=train_df['fold_id'], scoring='neg_mean_absolute_error', return_train_score=True, return_estimator=True)
# scores = scores['test_score'].mean()

print(scores)

{'fit_time': array([0.00797582, 0.00819016]), 'score_time': array([0., 0.]), 'estimator': [SVR(C=1000, degree=6, epsilon=0.36774550650687715), SVR(C=1000, degree=6, epsilon=0.36774550650687715)], 'test_score': array([-3.38370963, -1.14753815]), 'train_score': array([-0.29940579, -0.27971159])}


In [62]:
# Train, test
regr.fit(X, y)
y_pred_train = regr.predict(X)
y_pred_test = regr.predict(X_test)

print("Train:", Trainer.calculate_metrics(true_values=y, pred_values=y_pred_train))
print("OOS:", Trainer.calculate_metrics(true_values=y_test, pred_values=y_pred_test))

print("-"*20)
# cross-validation
cv_indices_dict = {0: [], 1: []}
index = 0
for _, row in train.iterrows():
    cv_indices_dict[row['fold_id']].append(index)
    index += 1
cv_indices = [[cv_indices_dict[0], cv_indices_dict[1]], [cv_indices_dict[1], cv_indices_dict[0]]]

amount_of_cv = 2
for cv_index in range(amount_of_cv):
    X_train_cv = X.iloc[cv_indices[cv_index][0]]
    y_train_cv = y.iloc[cv_indices[cv_index][0]]

    regr.fit(X_train_cv, y_train_cv)
    y_pred_train = regr.predict(X_train_cv)

    X_test_cv = X.iloc[cv_indices[cv_index][1]]
    y_test_cv = y.iloc[cv_indices[cv_index][1]]

    y_pred = regr.predict(X_test_cv)

    print(f"CV: {cv_index} Train:", Trainer.calculate_metrics(true_values=y_train_cv, pred_values=y_pred_train))
    print(f"CV: {cv_index} OOS:", Trainer.calculate_metrics(true_values=y_test_cv, pred_values=y_pred))



Train: {'mse': 0.098, 'mae': 0.291, 'r^2': 0.985}
OOS: {'mse': 0.722, 'mae': 0.63, 'r^2': 0.874}
--------------------
CV: 0 Train: {'mse': 0.106, 'mae': 0.31, 'r^2': 0.984}
CV: 0 OOS: {'mse': 0.882, 'mae': 0.744, 'r^2': 0.862}
CV: 1 Train: {'mse': 0.103, 'mae': 0.301, 'r^2': 0.984}
CV: 1 OOS: {'mse': 1.321, 'mae': 0.922, 'r^2': 0.805}


PLAIN:
Train: {'mse': 0.518, 'mae': 0.442, 'r^2': 0.921}
OOS: {'mse': 0.816, 'mae': 0.67, 'r^2': 0.858}
--------------------
CV: 0 Train: {'mse': 0.645, 'mae': 0.524, 'r^2': 0.905}
CV: 0 OOS: {'mse': 1.098, 'mae': 0.759, 'r^2': 0.828}
CV: 1 Train: {'mse': 0.248, 'mae': 0.285, 'r^2': 0.961}
CV: 1 OOS: {'mse': 1.167, 'mae': 0.799, 'r^2': 0.828}




MINMAX:
Train: {'mse': 0.195, 'mae': 0.231, 'r^2': 0.97}
OOS: {'mse': 0.545, 'mae': 0.536, 'r^2': 0.905}
--------------------
CV: 0 Train: {'mse': 0.246, 'mae': 0.25, 'r^2': 0.964}
CV: 0 OOS: {'mse': 0.841, 'mae': 0.718, 'r^2': 0.868}
CV: 1 Train: {'mse': 0.097, 'mae': 0.143, 'r^2': 0.985}
CV: 1 OOS: {'mse': 0.815, 'mae': 0.659, 'r^2': 0.88}



STANDART SCALER:
Train: {'mse': 0.077, 'mae': 0.122, 'r^2': 0.988}
OOS: {'mse': 0.483, 'mae': 0.554, 'r^2': 0.916}
--------------------
CV: 0 Train: {'mse': 0.075, 'mae': 0.111, 'r^2': 0.989}
CV: 0 OOS: {'mse': 0.655, 'mae': 0.642, 'r^2': 0.897}
CV: 1 Train: {'mse': 0.012, 'mae': 0.039, 'r^2': 0.998}
CV: 1 OOS: {'mse': 1.003, 'mae': 0.752, 'r^2': 0.852}




SCALE EVERYTHING STANDART SCALER
Train: {'mse': 0.098, 'mae': 0.291, 'r^2': 0.985}
OOS: {'mse': 0.722, 'mae': 0.63, 'r^2': 0.874}
--------------------
CV: 0 Train: {'mse': 0.106, 'mae': 0.31, 'r^2': 0.984}
CV: 0 OOS: {'mse': 0.882, 'mae': 0.744, 'r^2': 0.862}
CV: 1 Train: {'mse': 0.103, 'mae': 0.301, 'r^2': 0.984}
CV: 1 OOS: {'mse': 1.321, 'mae': 0.922, 'r^2': 0.805}